In [ ]:
#%%

from music21 import *
import copy, random

class BassModifier:
    def __init__(self, score_path: str):
        '''
        Takes as input a path to a score which contains the chord labels
        '''
        self.sc = converter.parse(score_path)
        self.bassline = self.sc.parts[1]
        self.blocks = {}
        self.notes = {n.offset: n for n in self.bassline.flatten()[note.Note]}
        self.measures = self.bassline[stream.Measure]

        for ch in self.bassline.flatten()[harmony.ChordSymbol]:
            ch.duration = self.notes[ch.offset].duration
            self.blocks[ch.offset] = ch
        print(self.blocks)
    
    def set_rhythm(self, n: note.Note, pattern: list[int]):
        '''
        Modifies a notes rhythm (where it is in a measure's context)
        '''
        d = n.quarterLength
        m = self.bassline.measure(n.measureNumber)
        curr_offset = n.offset # - m.offset
        # print("Starting at", offset, m.offset, n.measureNumber)
        denom = sum(pattern)
        for ix, amt in enumerate(pattern):
            new_d = amt / denom * d
            # print("Inserting", new_d, "at", curr_offset)
            if ix == 0:
                n.quarterLength = new_d
            else:
                new_n = note.Note(n.pitch, duration=duration.Duration(new_d))
                m.insert(curr_offset, new_n)
            curr_offset += new_d

    def inject_rhythm(self):
        '''
        For a loop of n chords, choose two patterns, one for first n-1, and a different
        one for the nth (to give rhythmic drive to the next loop)
        
        Currently, breaks into half-note divisions and provides one of the following
        distributions: 
        3:3:2, 3:1, 2:2, 1:1:1:1, 2:1:1, 1:1:2
        
        Operate on each half note first. Then organize which ones are special

        Todo: make more decisions on the weighting.
        '''
        # n = len(self.bassline.measures())
        # First pattern for the first n-1 measures
        weights = [4, 3, 3, 1, 3, 2]
        choices = [
            [3, 3, 2],
            [3, 1],
            [1, 1],
            [1, 1, 1, 1],
            [2, 1, 1],
            [1, 1, 2]
        ]
        selected_patterns = random.choices(choices, weights=weights, k=3)
        print(selected_patterns)
        
        for m in self.measures[:-1]:
            for n in m[note.Note]:
                # Divide it by 2
                self.set_rhythm(n, [2, 1, 1])
            for n in m[note.Note]:
                self.set_rhythm(n, selected_patterns[0])

        for n in self.measures[-1][note.Note]:
            self.set_rhythm(n, selected_patterns[2])

    
    def arpeggiate(self):
        '''
        Set more diverse pitches for the rhythms
        '''
        for m in self.measures[:-1]:
            for n in m[note.Note]:
                # Divide it by 2
                self.set_rhythm(n, [2, 1, 1])
            for n in m[note.Note]:
                self.set_rhythm(n, selected_patterns[0])

        for n in self.measures[-1][note.Note]:
            self.set_rhythm(n, selected_patterns[2])
    
    def thicken(self):
        '''
        Expand the notes to contain more chords, upwards
        '''
    
    def insert_approach_tones(self):
        '''
        Override some of the previous rhythms with approach tones in front of new chords
        '''
        # for offset, ch

    '''
    First pass: Adding rhythm 
    Split into half notes. Then, choose patterns for each quarter
    
    Choose two patterns and place one for the final transition to give energy
    
    Second pass: Adding pitch 
    Choosing the arpeggios
    
    Third pass: Extending chords upward
    '''
    
    

#%%

score_dir = '/Users/derrick/PycharmProjects/MIT/21M_383/music-melody-generator/scores/'
bm = BassModifier(score_dir + "Touch.mxl")

# list(bv.bassline.flatten()[note.Note].getElementsByOffset(4))[0].duration

bm.inject_rhythm()
bm.bassline.show()


#%%
